In [1]:
import pandas as pd
import numpy as np
import random

data=pd.read_csv(r'C:\Users\Reyan\Desktop\Proj\Apple\Data.csv')
data=data.sample(frac=1).reset_index(drop=True)

train_input_data=data.drop(['O'],axis=1)
train_input=np.matrix(train_input_data.values)
train_output_data=data.drop(['A','B','C','D'],axis=1)
train_output=np.matrix(train_output_data.values)
train_input[0].shape

(1, 4)

In [2]:
#set nn structure [input nodes,x hiddenlayers with x nodes....,output nodes, o for offset]
#function to generate random chromosomes
def chromosome_generetor(structure):
    x=[]
    for i in range(0,len(structure)-1):
        k=structure[i]*structure[i+1]
        x.append(k)
    x=sum(x)
    # x = shape of chromsome
    return 2*np.random.random((x,))-1 

#function to arrange the weights for each layers 
def weights_classifier(structure,chromosome):
    weights_list=[]
    for i in range(0,len(structure)-2):
        if i == 0:
            a=0
            b=structure[i]*structure[i+1]
        else:
            a=b
            b=a+(structure[i]*structure[i+1])
        r=structure[i]
        c=structure[i+1]
        #print('[',a,',',b,']','(',r,',',c,')')
        #print(len(chromosome))
        w=np.reshape(np.matrix(chromosome[a:b]),(r,c))
        
        weights_list.append(w)
    #print('weights: ',len(weights_list))   
    # weights_list = a list of aranged weights matrices  
    return weights_list

def sigmoid(x):
    return 1/(1+np.exp(-x))

def mse(train_output,answers):
    error=[]
    for i in range(len(answers)):
        mse=(train_output[i]-answers[i])**2
        error.append(mse)
    return error

def feed_forward(structure,chromosome,train_input):
    weights_list=weights_classifier(structure,chromosome)
    #for i in range(len(weights_list)):
        #print('weights list ',i,': ',weights_list[i].shape)
    for i in range(len(weights_list)):
        if i==0:
            output=sigmoid(np.dot(train_input,weights_list[i]))
        else:
            output=sigmoid(np.dot(output,weights_list[i]))
        #print('hl',[i],': ',weights_list[i])
    #print(output.shape)        
    return output
 
def learning(structure,chromosome,train_input):
    answers=[]
    for i in range(len(train_input)):
        output=feed_forward(structure,chromosome,train_input[i])
        answers.append(output)
    return np.array(answers)

In [3]:
population=[]

def populate(size):
    
    initial_pop=[None]*size
    
    for i in range(size):
        chromosome=chromosome_generetor(structure)
        initial_pop[i]=chromosome
    
    return initial_pop

def fitness(population,target,size,train_input,train_output,structure):
    
    score=[None]*size
    for index,chromosome in enumerate(population):
        
        answers=learning(structure,chromosome,train_input)
        error=mse(train_output,answers)
        score[index]=np.mean(error)-target,chromosome,index
        
    top_score=np.array(score)[np.array(score)[:,0].argsort()]
    #print(top_score)
    order=[None]*size
    for i in range(size):
        order[i]=top_score[i][1]
    
    top_snakes=np.array(order)[:int(size*0.1)]
    warrior=top_score[0]
    return top_snakes,warrior,answers


In [4]:
def mutation(child,m_rate):
    
    loop=int(round((len(child)/100)*m_rate))
    #x=len(child)
    #child=np.reshape(child,(1,x))
    x=child.tolist()
    #print(len(x))
    for _ in range(loop):
        i=random.randrange(0,len(x),1)
        #print(i)
        child[i]=child[i]+(2*np.random.random()-1)
    
    #child=np.reshape(child,(x,1))
    return child

def mate(population,size,m_rate,pattern):
    
    new_pop=[]
    for _ in range(int(size/2)):
        
        #print(len(population))
        p1=population[np.random.randint(0,len(population))]
        p2=population[np.random.randint(0,len(population))]
        

        if pattern=='single_point':
            x=int(round(((len(p1)/100)*50)+0.1))
            for i in range(2):
            
                if i == 0:
                    child = np.concatenate(([p1[:x],p2[x:]]))

                if i == 1:
                    child = np.concatenate(([p2[:x],p1[x:]]))
                
                child=mutation(np.array(child),m_rate)
                new_pop.append(child)
        
        if pattern=='multi_point':
            child=[None]*len(p1)
            for i in range(0,len(child),2):
                if i == len(child)-1:
                    child[i]=p1[i]
                else:
                    child[i]=p1[i]
                    child[i+1]=p2[i+1]       
            child=mutation(np.array(child),m_rate)
            new_pop.append(child)

            child=[None]*len(p1)
            for i in range(0,len(child),2):
                if i == len(child)-1:
                    child[i]=p2[i]
                else:
                    child[i]=p2[i]
                    child[i+1]=p1[i+1]
            child=mutation(np.array(child),m_rate)
            new_pop.append(child)
            
    return new_pop


def cycle(generation,population,target,size,train_input,train_output,m_rate,structure,pattern):
    population=populate(size)
    #print(len(population))
    gen=0
    for _ in range(generation):
        gen+=1
        population,warrior,answers=fitness(population,target,size,train_input,train_output,structure)
        print('Generation: ',gen,'-----','Fittest: ',warrior[0])
        population=mate(population,size,m_rate,pattern)
        #print(population[0])

    return population,answers

structure=[4,3,2,1,0]
size=1000
target=0
m_rate=100
pattern='single_point'
population,answers=cycle(100,population,target,size,train_input,train_output,m_rate,structure,pattern)

for i in range(len(answers)):
    print(np.round(answers[i],3),'===',train_output[i])

Generation:  1 ----- Fittest:  0.20858804061690295
Generation:  2 ----- Fittest:  0.19639484235910915
Generation:  3 ----- Fittest:  0.18922441495513248
Generation:  4 ----- Fittest:  0.15503839004231165
Generation:  5 ----- Fittest:  0.16289940751140403
Generation:  6 ----- Fittest:  0.15364842527667957
Generation:  7 ----- Fittest:  0.14617968783402954
Generation:  8 ----- Fittest:  0.13888850501456865
Generation:  9 ----- Fittest:  0.12922650224236026
Generation:  10 ----- Fittest:  0.11963771411521468
Generation:  11 ----- Fittest:  0.1166486880699146
Generation:  12 ----- Fittest:  0.09350985795064615
Generation:  13 ----- Fittest:  0.10672109628282517
Generation:  14 ----- Fittest:  0.10127443484198402
Generation:  15 ----- Fittest:  0.09825642726056699
Generation:  16 ----- Fittest:  0.09200862770981649
Generation:  17 ----- Fittest:  0.08133986294837131
Generation:  18 ----- Fittest:  0.07859444606396246
Generation:  19 ----- Fittest:  0.03982257046163745
Generation:  20 ----- 

In [6]:
structure=[4,3,2,1,0]
size=1000
target=0
m_rate=100
pattern='multi_point'
population,answers=cycle(100,population,target,size,train_input,train_output,m_rate,structure,pattern)

for i in range(len(answers)):
    print(np.round(answers[i],3),'===',train_output[i])

Generation:  1 ----- Fittest:  0.20925526335225178
Generation:  2 ----- Fittest:  0.19019002798249968
Generation:  3 ----- Fittest:  0.17979420305367877
Generation:  4 ----- Fittest:  0.16538346415816357
Generation:  5 ----- Fittest:  0.14256206247042644
Generation:  6 ----- Fittest:  0.140425059348172
Generation:  7 ----- Fittest:  0.1245290464599338
Generation:  8 ----- Fittest:  0.1097899998574202
Generation:  9 ----- Fittest:  0.10903507249707356
Generation:  10 ----- Fittest:  0.10597451838391297
Generation:  11 ----- Fittest:  0.09630752708633936
Generation:  12 ----- Fittest:  0.0903972703635692
Generation:  13 ----- Fittest:  0.06281123671906472
Generation:  14 ----- Fittest:  0.0482134477769039
Generation:  15 ----- Fittest:  0.0414663278869057
Generation:  16 ----- Fittest:  0.020652938550159865
Generation:  17 ----- Fittest:  0.024157022487550965
Generation:  18 ----- Fittest:  0.012129154863048587
Generation:  19 ----- Fittest:  0.010800797572805191
Generation:  20 ----- Fi